In [33]:
df1 = pd.read_parquet("data/output/test_fixed/uniprot_P35354/P35354_chembl_activities.parquet")
print(list(df1.columns))

['action_type', 'activity_comment', 'activity_id', 'activity_properties', 'assay_chembl_id', 'assay_description', 'assay_type', 'assay_variant_accession', 'assay_variant_mutation', 'bao_endpoint', 'bao_format', 'bao_label', 'canonical_smiles', 'data_validity_comment', 'data_validity_description', 'document_chembl_id', 'document_journal', 'document_year', 'ligand_efficiency', 'molecule_chembl_id', 'molecule_pref_name', 'parent_molecule_chembl_id', 'pchembl_value', 'potential_duplicate', 'qudt_units', 'record_id', 'relation', 'src_id', 'standard_flag', 'standard_relation', 'standard_text_value', 'standard_type', 'standard_units', 'standard_upper_value', 'standard_value', 'target_chembl_id', 'target_organism', 'target_pref_name', 'target_tax_id', 'text_value', 'toid', 'type', 'units', 'uo_units', 'upper_value', 'value', 'assay_description_enriched', 'assay_type_description', 'assay_organism_enriched', 'assay_strain', 'assay_tissue_enriched', 'assay_cell_type_enriched', 'assay_subcellular_

In [1]:
import pandas as pd

In [4]:
parquet = pd.read_parquet("data/annotation_table_with_uniprot.parquet")


0    [O43837]
1    [Q9X1A7]
2    [Q9X1A7]
3    [Q9X1A7]
4    [Q9X1A7]
Name: uniprot_ids, dtype: object

In [8]:
parquet[['system_id', 'uniprot_ids']].head()
test_parquet = parquet[:2].to_parquet("data/test_retrieval.parquet")

In [19]:
parquet[['system_id', 'uniprot_ids']].head()


,system_id,uniprot_ids
0,8grd__1__1.A_1.B__1.D,[O43837]
1,2grj__1__1.A__1.I,[Q9X1A7]
2,2grj__2__1.B__1.K,[Q9X1A7]
3,2grj__3__1.C__1.N,[Q9X1A7]
4,2grj__4__1.D__1.P,[Q9X1A7]


In [10]:
parquet[:2][['system_id', 'uniprot_ids']]

,system_id,uniprot_ids
0,8grd__1__1.A_1.B__1.D,[O43837]
1,2grj__1__1.A__1.I,[Q9X1A7]


In [30]:
parquet.columns['molregno']

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [26]:
"""
python retrieve_chembl_parallel_enriched.py \
  --input data/test_retrieval.parquet \
  --output data/output/chembl_retrieval_test \
  --workers 1 \
  --chunk-size 2 \
  --max-activities 500 \
  --skip-merge
"""

In [ ]:
df1 = pd.read_parquet("data/output/test_fixed/uniprot_P35354/P35354_chembl_activities.parquet")
print(df1['canonical_smiles'].nunique())
df1['canonical_smiles'].head()

455


0             Cc1ccc(-c2ccc(S(C)(=O)=O)cc2)n1-c1ccccc1
1     Cc1c(C=O)cc(-c2ccc(S(C)(=O)=O)cc2)n1-c1ccc(F)cc1
2    Cc1c(COc2cccc(Cl)c2)cc(-c2ccc(S(C)(=O)=O)cc2)n...
3       Fc1ccc(-c2[nH]c(-c3ccc(F)cc3)c3c2C2CCC3CC2)cc1
4         CCc1ccc(-c2ccc(S(C)(=O)=O)cc2)n1-c1ccc(F)cc1
Name: canonical_smiles, dtype: object

In [24]:
df2 = pd.read_parquet("data/output/test_fixed/uniprot_P00533/P00533_chembl_activities.parquet")
print(df2['canonical_smiles'].nunique())
df2['canonical_smiles'].head()

350


0    Cc1cc(C)c(/C=C2\C(=O)Nc3ncnc(Nc4ccc(F)c(Cl)c4)...
1    Cc1cc(C)c(/C=C2\C(=O)Nc3ncnc(Nc4ccc(F)c(Cl)c4)...
2    Cc1cc(C)c(/C=C2\C(=O)Nc3ncnc(Nc4ccc(F)c(Cl)c4)...
3    Cc1cc(C(=O)N2CCOCC2)[nH]c1/C=C1\C(=O)Nc2ncnc(N...
4    Cc1cc(C(=O)N2CCOCC2)[nH]c1/C=C1\C(=O)Nc2ncnc(N...
Name: canonical_smiles, dtype: object

## Split parquet into 2 parts for parallel processing on different computers

In [27]:
# Load the full annotation table
full_df = pd.read_parquet("data/annotation_table_with_uniprot.parquet")
print(f"Total systems: {len(full_df)}")

# Split into 2 equal parts
mid_point = len(full_df) // 2
part1 = full_df.iloc[:mid_point]
part2 = full_df.iloc[mid_point:]

print(f"\nPart 1: {len(part1)} systems (rows 0 to {mid_point-1})")
print(f"Part 2: {len(part2)} systems (rows {mid_point} to {len(full_df)-1})")

# Save the two parts
part1.to_parquet("data/annotation_table_with_uniprot_part1.parquet", index=False)
part2.to_parquet("data/annotation_table_with_uniprot_part2.parquet", index=False)

print("\n✓ Created:")
print("  - data/annotation_table_with_uniprot_part1.parquet")
print("  - data/annotation_table_with_uniprot_part2.parquet")

Total systems: 114537

Part 1: 57268 systems (rows 0 to 57267)
Part 2: 57269 systems (rows 57268 to 114536)

✓ Created:
  - data/annotation_table_with_uniprot_part1.parquet
  - data/annotation_table_with_uniprot_part2.parquet


### Run on each computer:

**Computer 1:**
```bash
python retrieve_chembl_parallel_enriched.py \
  --input data/annotation_table_with_uniprot_part1.parquet \
  --output data/chembl_parallel_enriched_part1 \
  --workers 8 \
  --chunk-size 50 \
  --max-activities 3000
```

**Computer 2:**
```bash
python retrieve_chembl_parallel_enriched.py \
  --input data/annotation_table_with_uniprot_part2.parquet \
  --output data/chembl_parallel_enriched_part2 \
  --workers 8 \
  --chunk-size 50 \
  --max-activities 3000
```

After both finish, you can merge the results or keep them separate.